In [8]:
import os
import pandas as pd
# pd.options.display.max_rows = None
from pulp import *

main_dir = os.getcwd()
output_dir = f"{main_dir}/outputs"

In [9]:
schedule = pd.read_csv(f"{output_dir}/schedule.csv")
schedule = schedule.rename(columns={"Unnamed: 7": "LocationDesignation"})
date_df = schedule.set_index('Date')
mile_matrix_wide = pd.read_csv(f"{output_dir}/mile_matrix_wide.csv")

mile_matrix_wide = mile_matrix_wide.rename(columns={"Unnamed: 0": "source"})
mile_matrix_long = mile_matrix_wide.melt(id_vars="source", var_name="destination", value_name="miles")

In [10]:
# ----- map mile matrix school names to the college football schedule -----
matrixlist_to_team = {
    'University of Illinois Urbana-Champaign': 'Illinois',
    'Indiana University Bloomington': 'Indiana',
    'University of Iowa': 'Iowa',
    'University of Maryland, College Park': 'Maryland',
    'University of Michigan': 'Michigan',
    'Michigan State University': 'MichiganState',
    'University of Minnesota Twin Cities': 'Minnesota',
    'University of Nebraska-Lincoln': 'Nebraska',
    'Northwestern University': 'Northwestern',
    'The Ohio State University': 'OhioState',
    'University of Oregon': 'Oregon',
    'Pennsylvania State University': 'PennState',
    'Purdue University': 'Purdue',
    'Rutgers, The State University of New Jersey, New Brunswick': 'Rutgers',
    'University of Southern California': 'SouthernCalifornia',
    'University of California, Los Angeles': 'UCLA',
    'University of Washington': 'Washington',
    'University of Wisconsin-Madison': 'Wisconsin'
}
mile_matrix_long['source'] = mile_matrix_long['source'].map(matrixlist_to_team)
mile_matrix_long['destination'] = mile_matrix_long['destination'].map(matrixlist_to_team)

In [11]:
# filter for only ohio state so it's always coming from Columbus
mile_matrix_long = mile_matrix_long[mile_matrix_long['source'] == "OhioState"]
mile_matrix_long = mile_matrix_long.dropna(subset=['destination', 'miles'])
mile_matrix_long = (
    mile_matrix_long
    .assign(source=lambda d: d["source"].str.strip(),
            destination=lambda d: d["destination"].str.strip())
    .drop_duplicates(["source", "destination"])
    .set_index(["source", "destination"])
    .sort_index()
)
distance_lookup = mile_matrix_long["miles"].to_dict()

In [12]:
# ----- clean the schedule -----
# rename columns
schedule.columns = ['Unnamed', 'Rk', 'Date', 'Time', 'Day', 'Winner', 'Pts', 'LocationDesignation', 'Loser', 'Pts.1', 'Notes']
# create home/away assignment
schedule['Home'] = schedule.apply(
    lambda row: row['Loser'] if row['LocationDesignation'] == '@' else row['Winner'],
    axis=1
)
schedule['Away'] = schedule.apply(
    lambda row: row['Winner'] if row['LocationDesignation'] == '@' else row['Loser'],
    axis=1
)

# assign points
schedule['HomePts'] = schedule.apply(
    lambda row: row['Pts.1'] if row['LocationDesignation'] == '@' else row['Pts'],
    axis=1
)
schedule['AwayPts'] = schedule.apply(
    lambda row: row['Pts'] if row['LocationDesignation'] == '@' else row['Pts.1'],
    axis=1
)

# select and order desired columns
schedule = schedule[['Rk', 'Date', 'Time', 'Day', 'Home', 'HomePts', 'Away', 'AwayPts', 'Notes']]
# clean the Home and Away column to remove rankings
schedule['Home'] = schedule['Home'].str.replace('\xa0', '', regex=False).str.replace(r"[ ()\d]", "", regex=True)
schedule['Away'] = schedule['Away'].str.replace('\xa0', '', regex=False).str.replace(r"[ ()\d]", "", regex=True)


In [14]:
# Define constants
DATES = [0, *list(schedule['Date'].unique())]
TEAMS = sorted(schedule['Home'].unique())
DESTINATIONS = [t for t in TEAMS if t != "OhioState"]

# Define LP problem
prob = LpProblem("OhioState_Travel", LpMinimize)

# Define decision variables
choices = LpVariable.dicts("Travel", (DATES, ['OhioState'], DESTINATIONS), cat='Binary')

# Each destination is visited exactly once
for t2 in DESTINATIONS:
    prob += lpSum([choices[d]['OhioState'][t2] for d in DATES[1:]]) == 1, f"Visit_{t2}"

# Only one trip allowed per date
for d in DATES[1:]:
    prob += lpSum([choices[d]['OhioState'][t2] for t2 in DESTINATIONS]) <= 1, f"OneTripPerDay_{d}"

# Ensure that travel only happens on valid home game dates for the destination
for d in DATES[1:]:
    for t2 in DESTINATIONS:
        if schedule[(schedule['Date'] == d) & (schedule['Home'] == t2)].shape[0] == 0:
            prob += choices[d]['OhioState'][t2] == 0, f"NoGame_{d}_{t2}"

# Objective: minimize total travel distance
total_distance = lpSum(
    choices[d]['OhioState'][t2] * distance_lookup[('OhioState', t2)]
    for d in DATES[1:]
    for t2 in DESTINATIONS
    if ('OhioState', t2) in distance_lookup
)
prob += total_distance, "MinimizeTravelDistance"
prob += total_distance <= 14000, "MaxTravelDistance"

# Solve
prob.solve()
print("Status:", LpStatus[prob.status])
print("Total Miles Traveled:", value(total_distance))

# Extract results
if LpStatus[prob.status] == 'Optimal':
    travel_schedule = []
    for d in DATES[1:]:
        for t2 in DESTINATIONS:
            if choices[d]['OhioState'][t2].varValue == 1:
                travel_schedule.append({'source': 'OhioState', 'destination': t2, 'date': d})

    solution_long = pd.DataFrame(travel_schedule)
    solution_long['date'] = pd.to_datetime(solution_long['date'], format='%b %d, %Y', errors='coerce')
    solution_long = solution_long.sort_values('date')
    solution_long.to_csv(f"{output_dir}/osu_outcome_travel_schedule.csv")
else:
    print("Failed to find an optimal solution.")

Status: Optimal
Total Miles Traveled: 12630.706757658263
